# BLEU-n 评估


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import numpy as np

# 加载预训练模型和 tokenizer
model_name = "C:/Users/Admin/Desktop/model/qwen/Qwen2___5-0___5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 确保 tokenizer 定义了填充标记
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# 加载评估数据集
data_files = {"test": "C:/Users/Admin/Desktop/Data/huatuo_train.json"}
dataset = load_dataset("json", data_files=data_files, split="test")

c:\Users\Admin\.conda\envs\bloom_ft\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = "介绍你自己"
messages = [
    {
        "role": "system",
        "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.",
    },
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

In [6]:
# 提取对话中的问题和参考值
def extract_conversations(dataset):
    questions = []
    references = []
    for item in dataset:
        for conversation in item["conversations"]:
            if conversation["from"] == "human":
                questions.append(conversation["value"])
            elif conversation["from"] == "gpt":
                references.append(conversation["value"])
    return questions, references


questions, references = extract_conversations(dataset)


def predict_responses(questions):
    responses = []
    for question in questions:
        messages = [
            {
                "role": "system",
                "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.",
            },
            {"role": "user", "content": question},
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
        )
        generated_ids = [
            output_ids[len(input_ids) :]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        responses.append(response)
    return responses


predictions = predict_responses(questions)


# 打印一些示例的参考答案和预测结果
for i in range(5):
    print(f"Question: {questions[i]}")
    print(f"Reference: {references[i]}")
    print(f"Prediction: {predictions[i]}")
    print()

# 定义平滑函数
smooth = SmoothingFunction().method4

# 计算语料库级别的BLEU
references = [[ref.split()] for ref in references]
candidates = [pred.split() for pred in predictions]

bleu_score = corpus_bleu(references, candidates, smoothing_function=smooth)
print(f"Corpus BLEU Score: {bleu_score:.4f}")

# 计算每个句子的BLEU（可选）
for i, (refs, can) in enumerate(zip(references, candidates)):
    sent_bleu = corpus_bleu([refs], [can], smoothing_function=smooth)
    print(f"Sentence {i+1} BLEU Score: {sent_bleu:.4f}")

c:\Users\Admin\.conda\envs\bloom_ft\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Question: 曲匹地尔片的用法用量
Reference: 注意：同种药品可由于不同的包装规格有不同的用法或用量。本文只供参考。如果不确定，请参看药品随带的说明书或向医生询问。口服。一次50～100mg（1-2片），3次/日，或遵医嘱。
Prediction: 曲匹地尔片是一种中药，主要用于治疗肝阳上亢、头痛眩晕等症状。其使用方法和剂量可能会因人而异，通常需要在医生指导下进行。

一般情况下，成人常用量为每次1-2片，每日3次，饭后服用。具体用量应根据个人情况调整，并且不要超过推荐剂量。请务必遵医嘱使用药物，不可擅自增减药量或停药。

请注意：以上信息仅供参考，实际用药时应遵循专业医师指导。如果对药品有任何疑问，请及时咨询医疗专业人士。

Question: 三期梅毒多久能治愈吗
Reference: 梅毒是一种进展十分缓慢的疾病，包括早期梅毒，晚期梅毒等等，一般来说，梅毒病程超过两年的就是晚期梅毒了，而晚期梅毒的治疗难度是比较大的，三期梅毒能完全治愈吗?该如何进行治疗?一、对于三期梅毒的治疗，中西各有妙招，希望患者在医生的指导下选择适合自己的治疗方法。被确诊为三期梅毒的患者很关心这个问题，梅毒需要讲究科学的治疗方法，三期梅毒患者如果有一个良好的心态，积极配合医生治疗，还有很有可能治愈的，具体治疗方法我们看看下文的介绍。二、三期梅毒又称晚期梅毒，是因梅毒初期没有进行治疗或者治疗不彻底所造成。好发于40-50岁之间。该期梅毒不仅局限于皮肤和粘膜，也可侵犯任何内脏器官和组织。三期梅毒可发生在感染后2年以上，一般多发于感染后3～4年。病程漫长，可持续10～30年。未经治愈的二期梅毒中约有1/3的病人可发展为晚期活动性梅毒;另有一部分患者不出现晚期梅毒症状，只是梅毒血清反应持续阳性，为晚期潜伏梅毒;也有一部分患者可以自愈。三、晚期梅毒可以通过药物治疗控制病情，但是治愈的可能性是比较小的，治疗还是有必要的，通过有效的治疗可以缓解患者的症状，杀灭一部分的梅毒螺旋体。虽然治愈的可能性不高，但是患者还是应当坚持治疗。通过上文的简单介绍，相信大家对于三期梅毒多久能治愈这一方面的问题已经有了自己的答案。对于梅毒患者来说，无论患病多少年，都应在确诊后及时进行正规的治疗处理。正规的治疗可以缓解患者的病情，患者还是应当坚持进行，并积极配合医生的治疗，说不定治愈的可能就发生了。
Predi